In [ ]:
import os
from dotenv import load_dotenv
from spark_common import MinIOSparkManager
from setup import LakehouseSetupManager
from pyspark.sql import Row

# 加载 .env 文件中的配置
load_dotenv()

# 从环境变量获取配置（若获取不到则使用默认值）
MINIO_ENDPOINT = os.getenv("AWS_ENDPOINT_URL", "http://minio:9000")
MINIO_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID", "admin")
MINIO_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY", "admin_password")
MINIO_BUCKET = os.getenv("MINIO_BUCKET", "reddit-warehouse")

# 1. 初始化 Manager (使用环境变量)
manager = MinIOSparkManager(
    endpoint=MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    bucket=MINIO_BUCKET
)

# 2. 启动 Spark
spark = manager.create_session("RedditAnalysisProject")



In [ ]:
# 3. 初始化 Setup Manager
# 注意：DBsetup 的 endpoint 也改为了 MINIO_ENDPOINT (9000 端口)
# 因为代码内部读写数据必须走 API 端口，而不是 Console 端口
DBsetup = LakehouseSetupManager(
    spark_session=spark,     
    endpoint=MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    bucket=MINIO_BUCKET
)


In [ ]:
# 4. 环境准备：清理并重新建表
print("🧹 正在清理并初始化环境...")
DBsetup.cleanup()

In [ ]:
DBsetup.setup()

In [ ]:
# 5. 验证表结构 
DBsetup.validate()

In [ ]:
# 任务完成
print("✅ Lakehouse 环境初始化成功")


In [ ]:
spark.stop()